In [1]:
import sys, os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.utils import img_to_array
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16

In [2]:
train_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/train"
test_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/test"
img_size = 255
num_classes = 3
class_names = ["Crazing", "Inclusion", "Patches"]

In [3]:
augment = keras.Sequential(
    [layers.RandomFlip("horizontal_and_vertical"), 
     layers.RandomRotation(0.2)]
)

2023-11-21 22:04:26.687175: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2023-11-21 22:04:26.687192: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-21 22:04:26.687196: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-21 22:04:26.687221: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-21 22:04:26.687233: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
data_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/"
INPUT_TFRECORD_TRAIN = os.path.join(data_dir, "train_tfrecords")
INPUT_TFRECORD_TEST = os.path.join(data_dir, "test_tfrecords")

In [53]:
class LoadTfrecords:

    def __init__(self, BATCH_SIZE):
        self.BATCH_SIZE = BATCH_SIZE


    def parse_train(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.uint8)
        labels = tf.reshape(labels, [3])

        return images, labels
    

    def parse_test(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.uint8)
        labels = tf.reshape(labels, [3])

        return images, labels


In [54]:
parse_data = LoadTfrecords(BATCH_SIZE=32)

trainset = tf.data.TFRecordDataset(INPUT_TFRECORD_TRAIN)
trainset = trainset.map(parse_data.parse_train)
trainset = trainset.shuffle(buffer_size=828)
trainset = trainset.repeat(-1)
trainset = trainset.batch(32)
trainset = trainset.prefetch(buffer_size=4)

testset = tf.data.TFRecordDataset(INPUT_TFRECORD_TEST)
testset = testset.map(parse_data.parse_test)
testset = testset.batch(32)
testset = testset.prefetch(buffer_size=2)

In [77]:
input = Input(shape=(200, 200, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input)

In [78]:
x = vgg16.output
x = Flatten(input_shape=vgg16.output_shape[1:])(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(3, activation='softmax')(x)
model = keras.models.Model(inputs=vgg16.input, outputs=prediction)
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0  

                                                                 
 dense_24 (Dense)            (None, 256)               4718848   
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_25 (Dense)            (None, 3)                 771       
                                                                 
Total params: 19434307 (74.14 MB)
Trainable params: 19434307 (74.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
vgg16.trainable = True
set_trainable = False
for layer in vgg16.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [80]:
for layer in vgg16.layers:
    print(layer, layer.trainable)

<keras.src.engine.input_layer.InputLayer object at 0x29a230b50> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x29a230fa0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2874b2af0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x2817750a0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x175bfa3a0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x281719a00> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x285f5da60> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x17f657100> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2816fe460> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x17e24e250> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x17e256640> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x17e2568e0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x17e2be7c0> False
<keras.sr

In [81]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.SGD(learning_rate=1e-5), 
              metrics=['acc'])

In [82]:
history = model.fit(trainset, 
                    steps_per_epoch=828//32, 
                    epochs=10, 
                    validation_data=testset, 
                    verbose=1)

Epoch 1/10


2023-11-21 23:28:09.216537: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.
2023-11-21 23:28:09.239161: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5585763089763447940


InvalidArgumentError: Graph execution error:

Detected at node Reshape defined at (most recent call last):
<stack traces unavailable>
Detected at node Reshape defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Input to reshape is a tensor with 40000 values, but the requested shape has 120000
	 [[{{node Reshape}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  Input to reshape is a tensor with 40000 values, but the requested shape has 120000
	 [[{{node Reshape}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_12837]

In [ ]:
print(vgg16.output_shape)

(None, 6, 6, 512)
